<a href="https://colab.research.google.com/github/nczarli/f-jl221-2/blob/main/soft_teacher_custom_dataset_resume_from_400_to_800.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load GitHub Repos

In [1]:
!git clone https://github.com/nczarli/f-jl221-2.git
!git clone https://github.com/nczarli/SoftTeacherCustomDataset.git

Cloning into 'f-jl221-2'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 214 (delta 122), reused 128 (delta 71), pack-reused 0
Receiving objects: 100% (214/214), 8.34 MiB | 6.49 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Cloning into 'SoftTeacherCustomDataset'...
remote: Enumerating objects: 383, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 383 (delta 164), reused 161 (delta 129), pack-reused 148
Receiving objects: 100% (383/383), 600.03 KiB | 6.19 MiB/s, done.
Resolving deltas: 100% (192/192), done.


# Install Requirements

In [2]:
ls

f-jl221-2/  sample_data/  SoftTeacherCustomDataset/


In [3]:
cd SoftTeacherCustomDataset/

/content/SoftTeacherCustomDataset


In [4]:
ls

configs/  LICENSE   README.md         resources/   setup.py  tools/
demo/     Makefile  requirements.txt  SECURITY.md  ssod/


In [5]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.4 MB/s eta 0:14:31tcmalloc: large alloc 1147494400 bytes == 0x3929a000 @  0x7fd2dd472615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:56tcmalloc: large alloc 1434370048 bytes == 0x7d8f0000 @  0x7fd2dd472615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |█████

In [6]:
pip install mmcv-full==1.3.17 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 50.4 MB 58 kB/s 
     |████████████████████████████████| 190 kB 11.6 MB/s 


In [7]:
!make install

make pre
make[1]: Entering directory '/content/SoftTeacherCustomDataset'
python -m pip install -r requirements.txt -f https://download.pytorch.org/whl/torch_stable.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 1.9 MB 16.5 MB/s 
     |████████████████████████████████| 182 kB 53.2 MB/s 
     |████████████████████████████████| 166 kB 67.3 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 13.7 MB/s 
     |████████████████████████████████| 162 kB 46.3 MB/s 
     |████████████████████████████████| 162 kB 60.6 MB/s 
     |████████████████████████████████| 158 kB 48.7 MB/s 
     |████████████████████████████████| 157 kB 53.8 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 79.3 MB/s 
     |█████████████████████

In [8]:
cd ..

/content


In [9]:
ls

f-jl221-2/  sample_data/  SoftTeacherCustomDataset/


# Copy Files into Working Dir

In [10]:
!cp -R f-jl221-2/labels_generated/ SoftTeacherCustomDataset/labels_generated

In [11]:
cd SoftTeacherCustomDataset/

/content/SoftTeacherCustomDataset


In [12]:
ls

configs/           LICENSE    requirements.txt  setup.py        thirdparty/
demo/              Makefile   resources/        ssod/           tools/
labels_generated/  README.md  SECURITY.md       ssod.egg-info/


## Preprocess images

In [13]:
# Preprocess the images, scale y axis by 1.5

# find all bmp images in labels_generated folder
# for each image resize y axis by 1.5 times and overwrite the image

import os
import cv2
import numpy as np

# get all bmp files in labels_generated and subfolders
bmp_files = []

for root, dirs, files in os.walk("labels_generated"):
    for file in files:
        if file.endswith(".bmp"):
             bmp_files.append(os.path.join(root, file))
print(len(bmp_files))
# resize each bmp file
for bmp_file in bmp_files:
    img = cv2.imread(bmp_file, cv2.IMREAD_GRAYSCALE)
    # print(img.shape)
    img = cv2.resize(img, (0,0), fx=1, fy=1.5)
    # print(img.shape)
    cv2.imwrite(bmp_file, img)

64


## Copy Resume From Model

In [14]:
# Resume from script
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
cd /content/SoftTeacherCustomDataset

/content/SoftTeacherCustomDataset


In [16]:
mkdir resume_from

In [17]:
# Copy chosen start to SoftTeacherCustomDataset
!cp -R /content/drive/MyDrive/f-jl221-2/soft_teacher_400_iter/iter_400.pth /content/SoftTeacherCustomDataset/resume_from

# Training

In [20]:
!bash tools/dist_train.sh configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume.py 1

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : tools/train.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 1
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_6p5q715s/none_vym13km0
INFO:torch.distributed.elastic.agent.server.api:[default] starti

# Inference

## Add paths of thirdparty packages

In [21]:
import sys
sys.path.insert(0,'/content/SoftTeacherCustomDataset/thirdparty/mmdetection/mmdet')

In [22]:
import sys
print(sys.path)

['/content/SoftTeacherCustomDataset/thirdparty/mmdetection/mmdet', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']


In [23]:
import sys
sys.path.insert(0,' /content/SoftTeacherCustomDataset')

In [24]:
!export PYTHONPATH=$(pwd)

In [25]:
cd /content/SoftTeacherCustomDataset/thirdparty/mmdetection/

/content/SoftTeacherCustomDataset/thirdparty/mmdetection


In [26]:
import mmdet

In [33]:
cd /content

/content


## Visualise

In [34]:
!python SoftTeacherCustomDataset/demo/image_demo.py /content/SoftTeacherCustomDataset/labels_generated/test/images/Cherries,24,19-08-2022,18-10-52,pass.bmp SoftTeacherCustomDataset/configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume.py SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume/latest.pth --output work_dirs/

load checkpoint from local path: SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume/latest.pth
/content/SoftTeacherCustomDataset/thirdparty/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Save results to work_dirs/Cherries,24,19-08-2022,18-10-52,pass.bmp


In [35]:
cd SoftTeacherCustomDataset/

/content/SoftTeacherCustomDataset


In [36]:
!bash tools/dist_test.sh configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume.py work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume/latest.pth 1 --eval bbox --cfg-options model.test_cfg.rcnn.score_thr=0.0001


/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : tools/test.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 1
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_ybab2q60/none_as5f8ua_
INFO:torch.distributed.elastic.agent.server.api:[default] startin

In [37]:
cd ..

/content


In [38]:
!python SoftTeacherCustomDataset/demo/image_demo.py '/content/SoftTeacherCustomDataset/labels_generated/unlabelled/images/*.bmp' SoftTeacherCustomDataset/configs/soft_teacher/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume.py SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume/iter_800.pth --output work_dirs/

load checkpoint from local path: SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume/iter_800.pth
/content/SoftTeacherCustomDataset/thirdparty/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Save results to work_dirs/300_pixels_Cherries,31,19-08-2022,18-17-23,pass.bmp
Save results to work_dirs/100_pixels_Cherries,3

In [39]:
!cp -R /content/work_dirs /content/drive/MyDrive/f-jl221-2/soft_teacher_400_to_800_iter

In [41]:
!cp -R /content/SoftTeacherCustomDataset/work_dirs/soft_teacher_faster_rcnn_r50_caffe_fpn_coco_generated_labels_resume/iter_800.pth /content/drive/MyDrive/f-jl221-2/soft_teacher_400_to_800_iter